In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import random
import numpy as np
import pandas as pd
from random import randint
import sys

In [3]:
ingredients_df = pd.read_csv('/content/drive/Shareddrives/genetic algorithm/codes/ingredients-dataset.csv')
df = ingredients_df.dropna()

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 0 to 199
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   No                200 non-null    int64  
 1   Bahan             200 non-null    object 
 2   Satuan (gr)       200 non-null    int64  
 3   Protein(gr)       200 non-null    float64
 4   Lemak(gr)         200 non-null    float64
 5   Karbohidrat(gr)   200 non-null    float64
 6   Protein(kal)      200 non-null    float64
 7   Lemak(kal)        200 non-null    float64
 8   Karbohidrat(kal)  200 non-null    float64
 9   Harga (IDR)       200 non-null    int64  
 10  total kalori      200 non-null    float64
dtypes: float64(7), int64(3), object(1)
memory usage: 18.8+ KB


In [5]:
DOMAIN = len(df)-1
LEN_CHROMOSOME = 15

In [6]:
def create_individual(N=DOMAIN, len_chromosome=LEN_CHROMOSOME):
    return [randint(0,N) for _ in range(len_chromosome)]

In [7]:
def get_population(n_population):
  population = [create_individual() for _ in range(n_population)]
  return population

In [8]:
def get_needs(age, weight, height, f_activity=1.7):
  amb = float(66.5 + (13.7 * weight) + (5 * height) - (6.8 * age))
  tee = float(amb * f_activity)

  # because these are still ingredients that need to be cooked,
  # we spare some space for the other ingredients
  const = 0.95 
  protein = float(0.25 * tee * const)
  fat = float(0.15 * tee * const)
  carb = float(0.60 * tee * const)

  return protein, fat, carb

In [9]:
def get_nutrition_and_price(individual):
  sum_price = sum_protein = sum_carb = sum_fat = 0

  for allele in individual:
    sum_protein += df.iloc[allele]['Protein(kal)']
    sum_fat += df.iloc[allele]['Lemak(kal)']
    sum_carb += df.iloc[allele]['Karbohidrat(kal)']
    sum_price +=  df.iloc[allele]['Harga (IDR)']

  return [sum_protein, sum_fat, sum_carb], sum_price

def get_fitness(individual, needs, alpha_protein=5, alpha_fat=5, alpha_carb=5):
  nutrition, price = get_nutrition_and_price(individual)
  protein, fat, carb = needs
  penalty_protein = float(abs(nutrition[0] - protein))
  penalty_fat = float(abs(nutrition[1] - fat))
  penalty_carb = float(abs(nutrition[2] - carb))
  
  sum_penalties = (alpha_protein*penalty_protein) + (alpha_fat*penalty_fat) + (alpha_carb*penalty_carb)

  fitness = 1/((price/1000) + sum_penalties)
  return fitness

In [10]:
def crossover(parent1, parent2, pc):
  if random.uniform(0.0, 1.0) < pc:
    point1 = randint(1, len(parent1)-3)
    point2 = randint(point1, len(parent1)-2)

    children1 = parent2[:point1] + parent1[point1:point2] + parent2[point2:]
    children2 = parent1[:point1] + parent2[point1:point2] + parent1[point2:]
    return children1, children2
  else:
    return parent1, parent2

In [11]:
def mutation(individual, pm):
  for allele in range(len(individual)):
    if random.uniform(0.0, 1.0) < pm:
      creep = randint(-3, 3)
      individual[allele] = individual[allele] + creep
      # change with random numbers if the mutation was outside the domain
      if (individual[allele]<0 or individual[allele]>DOMAIN):
        individual[allele] = randint(0,DOMAIN)
  return individual

In [12]:
def genetic_algorithm(needs, n_iter, n_population, pc, pm):
  population = get_population(n_population)
  
  best_fits = list()

  best_individual, best_fit = population[0], get_fitness(population[0], needs)
  best_fits.append(best_fit)
  
  for generation in range(n_iter):
    fitness = [get_fitness(individual, needs) for individual in population]
    for individual in range(n_population):
      if fitness[individual] > best_fit:
        best_individual, best_fit = population[individual], fitness[individual]
        best_fits.append(best_fit)
    print(f'{generation+1}-th, best fit: {population[individual]}, fitness: {fitness[individual]}')
    parents = population
    children = list()
    for i in range(0, n_population, 2):
      parent1, parent2 = parents[i], parents[i+1]
      for individual in crossover(parent1, parent2, pc):
        mutation(individual, pm)
        children.append(individual)
    population = children

  best_fitness_mean = sum(best_fits)/len(best_fits)
  return best_individual, best_fit, best_fitness_mean

In [13]:
def exhaustive_search(needs):
  n_population_params = [10, 30, 50]
  n_iteration = 100
  pc_params = [0.8, 0.85, 0.9]
  pm_params = [0.0002, 0.067]

  best_n_population = n_population_params[0]
  best_pc = pc_params[0]
  best_pm = pm_params[0]

  best_individual, best_fit, best_fitness_mean = genetic_algorithm(needs, n_iteration, best_n_population, best_pc, best_pm)

  for n_pop_param in n_population_params:
      for pc_param in pc_params:
        for pm_param in pm_params:
            individual, fit, fitness_mean = genetic_algorithm(needs, n_iteration, n_pop_param, pc_param, pm_param)
            if fit > best_fit:
              best_individual = individual

              best_fit = fit
              best_fitness_mean = fitness_mean
              best_n_population = n_pop_param
              best_pc = pc_param
              best_pm = pm_param

  return best_individual, best_fit, best_fitness_mean, best_n_population, n_iteration, best_pc, best_pm


In [14]:
age = 23
weight = 70
height = 175
activity = "ringan"    
ACTIVITY_MAP = {'istirahat_total': 1,
             'sangat_ringan': 1.3,
             'ringan': 1.6,
             'sedang': 1.7,
             'berat': 2.1,
             'sangat_berat': 2.4}

In [15]:
needs = get_needs(age, weight, height, ACTIVITY_MAP[activity])

In [16]:
best_individual, best_fit, best_fitness_mean, best_n_population, n_iteration, best_pc, best_pm = exhaustive_search(needs)

1-th, best fit: [50, 118, 108, 59, 104, 98, 196, 30, 161, 17, 121, 76, 153, 139, 132], fitness: 0.00024983654444079976
2-th, best fit: [50, 118, 108, 59, 104, 98, 196, 30, 161, 17, 121, 76, 153, 139, 132], fitness: 0.00024983654444079976
3-th, best fit: [69, 155, 108, 59, 104, 98, 196, 21, 94, 33, 180, 7, 25, 115, 191], fitness: 0.00015115138052601893
4-th, best fit: [50, 118, 116, 141, 58, 108, 176, 21, 94, 33, 180, 7, 25, 139, 132], fitness: 0.0001561575010802195
5-th, best fit: [69, 155, 108, 59, 104, 98, 196, 21, 161, 17, 121, 76, 153, 115, 191], fitness: 0.00022137656374870228
6-th, best fit: [69, 155, 108, 59, 104, 98, 196, 21, 161, 17, 121, 76, 153, 115, 191], fitness: 0.00022137656374870228
7-th, best fit: [50, 155, 108, 59, 104, 98, 196, 21, 161, 17, 121, 7, 25, 139, 132], fitness: 0.00018692378407013087
8-th, best fit: [50, 155, 108, 59, 104, 98, 196, 21, 161, 17, 121, 7, 25, 139, 132], fitness: 0.00018692378407013087
9-th, best fit: [69, 118, 116, 141, 58, 108, 176, 30, 94, 

In [17]:
print(f'got best parameter n_population: {best_n_population}, n_iteration: {n_iteration}, best_pc: {best_pc}, best_pm: {best_pm}')

got best parameter n_population: 50, n_iteration: 100, best_pc: 0.9, best_pm: 0.067


In [18]:
print(f'with best_fitness_mean: {best_fitness_mean}')

with best_fitness_mean: 0.0014203493871496732


In [19]:
print(f'best individual: {best_individual}, with fitness: {best_fit}')

best individual: [53, 190, 13, 123, 176, 81, 8, 88, 3, 177, 150, 60, 85, 28, 91], with fitness: 0.004266903337571815


In [20]:
ingredients = df.iloc[best_individual]
ingredients

,No,Bahan,Satuan (gr),Protein(gr),Lemak(gr),Karbohidrat(gr),Protein(kal),Lemak(kal),Karbohidrat(kal),Harga (IDR),total kalori
53,53,Ikan kakap,100,20.0,0.7,0.0,80.0,6.3,0.0,9500,86.3
190,190,Talas bogor,200,2.8,0.8,50.0,11.2,7.2,200.0,4200,218.4
13,13,Kentang segar,100,2.1,0.2,33.7,8.4,1.8,134.8,1860,145.0
123,123,Lepok,100,2.8,1.2,39.8,11.2,10.8,159.2,25000,181.2
176,176,Daun ndusuk,100,1.7,0.5,38.8,6.8,4.5,155.2,5000,166.5
81,81,Ikan tongkol,100,13.7,1.5,8.0,54.8,13.5,32.0,5300,100.3
8,8,Kacang kedelai segar,100,30.2,15.6,30.1,120.8,140.4,120.4,2900,381.6
88,88,Umbut kelapa,100,2.0,0.2,7.2,8.0,1.8,28.8,5500,38.6
3,3,Beras ketan hitam,100,8.0,2.3,74.5,32.0,20.7,298.0,1750,350.7
177,177,Ikan kembung,100,21.3,3.4,2.2,85.2,30.6,8.8,3800,124.6


In [21]:
protein, fat, carb = needs
print(f'protein: {protein}, fat: {fat}, carbohidrat:{carb}')

protein: 662.7579999999999, fat: 397.65479999999997, carbohidrat:1590.6191999999999


In [22]:
ingredients['Karbohidrat(kal)'].sum()

1582.4

In [23]:
ingredients['Lemak(kal)'].sum()

397.79999999999995

In [24]:
ingredients['Protein(kal)'].sum()

644.8000000000001

In [25]:
ingredients['Harga (IDR)'].sum()

102750